# Listando todos os arquivos nos sub-diretórios

Para calcular a média de cada eletrodo em cada arquivo, é necessário recuperar o nome de cada arquivos nas sub-pastas.



In [31]:
import os

def get_sub_files(path_name, files=[]):

    subpaths = os.listdir(path_name)

    for path in subpaths:
        full_name = path_name + path
        if os.path.isdir(full_name):
            get_sub_files(full_name + '/', files) # goes to next sub-level and concat with the current list of files
        elif not path.startswith('.'):
            files.append(full_name)

    return files

all_files = get_sub_files('dataset/')

# Carregando os dados dos eletrodos

Com todos os arquivos listados, basta abrir cada um e salvar os valores de cada eletrodo. Se dois arquivos contêm os eletrodos, por exemplo, `P17`, os seus valores serão somados e calculados a média.

In [37]:
# apenas retorna de qual pasta esse arquivo é
def pathName(fileName):
    if fileName.__contains__('small'): return 'small'
    if fileName.__contains__('full'): return 'full'
    if fileName.__contains__('large_train'): return 'large_train'
    if fileName.__contains__('large_test'): return 'large_test'
    return 'none'

# se a pasta ainda não tiver nenhum mapeamento, cria
def verifyPath(output, path):
    try:
        output[path]
    except KeyError:
        output[path] = {}

# dada o mapeamento da pasta, o eletrodo e o valor,
# apenas insere este valor
def insert(outputPath, electrode, pos, value):
    try:
        outputPath[electrode][pos] += value
    except KeyError:
        outputPath[electrode] = list(range(256))
        insert(outputPath, electrode, pos, value)
    #try:
    #    output[line[1]][int(line[2])] += float(line[3])
    #except KeyError:
    #    output[line[1]] = list(range(256))
    #    output[line[1]][int(line[2])] = float(line[3])

def process_file(output, file, path):

    verifyPath(output, path)

    for line in file.readlines():
        if line.startswith('#'):
            continue

        # 41 FP2 168 -4.720
        line = line.split()
        electrode, pos, value = line[1:4]
        insert(output[path], electrode, int(pos), float(value))

def process_files(files, output={}):

    pastPath = ''
    files = files[::-1]
    for file in files:
        currentPath = pathName(file)

        if not currentPath.__eq__(pastPath):
            print(currentPath)
            pastPath = currentPath

        process_file(output, open(file), currentPath)

    return output

output = process_files(all_files)

small
large_train


ValueError: not enough values to unpack (expected 3, got 0)

# Salvando os dados

Com todos os dados dos eletrodos, bastas salvar esse objeto em `.json`.

In [33]:
import json

fileOutput = open('result.json', 'w')
json.dump(output, fileOutput)
fileOutput.close()